# Catalogues exploration
Ce notebook a pour but d'analyser et de rassembler les diffrentes sources de données afin de constituer un catalogue qui se veut être assez exhaustif accessible via une Web API.  

## 1. OpenNGC catalogue
 Ce catalogue a été télécharger depuis [ce lien](https://github.com/mattiaverga/OpenNGC). Il contient toutes les données présentes dans le catalogue NGC.

In [36]:
#import sys
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
CHEMIN_SOURCES = os.path.join("..","1_Data_brute")

In [2]:
# Lecture de données depuis le fichier source vers un objet DataFrame
col_names = ['NGC_IC_designation', 
    'object_type',
    'Right_Ascension',
    'Declinaison',
    'Constelation_name_abrv',
    'major_axis',
    'minor_axis',
    'major__axis_position_angle', 
    'B_Apparent_Magnitude',
    'V_Apparent_Magnitude',             
    'J_Apparent_Magnitude',
    'H_Apparent_Magnitude',
    'K_Apparent_Magnitude',
    'Mean_surface_brigthness', 
    'Hubble_morphological_type',
    'center_star_U_maghitude',
    'center_star_B_maghitude',
    'center_star_V_maghitude',
    'Messier_number',
    'NGC_numner',
    'IC_number',
    'Center_star_name',
    'Identifiers',
    'Common_names',       
    'NED_notes',  
    'OpenNGC_notes']

NGC_mattiaAverga_GitHub = pd.read_csv(os.path.join(CHEMIN_SOURCES, "NGC_mattiaAverga_GitHub.csv"), sep=';',header=0, names=col_names)
NGC_mattiaAverga_GitHub.shape

(13958, 26)

Dans cette [source](http://messier.obspm.fr/m-q&a.html), on peut lire:
- M24: the Milky Way Patch in Sagittarius; it contains however the 11th magnitude cluster NGC 6603 which is sometimes erroneously listed as M24. Also, it may be that IC 4715 is M24. 
- M25: this is IC 4725, 
- M40: the double star Winnecke 4
- M45: the Pleiades; however, this cluster is associated with nebulae which have NGC numbers.

In [6]:
NGC_mattiaAverga_GitHub.loc[(NGC_mattiaAverga_GitHub['NGC_IC_designation']=='IC4725') |
                        (NGC_mattiaAverga_GitHub['NGC_IC_designation']=='IC4715')]

,NGC_IC_designation,object_type,Right_Ascension,Declinaison,Constelation_name_abrv,major_axis,minor_axis,major__axis_position_angle,B_Apparent_Magnitude,V_Apparent_Magnitude,J_Apparent_Magnitude,H_Apparent_Magnitude,K_Apparent_Magnitude,Mean_surface_brigthness,Hubble_morphological_type,center_star_U_maghitude,center_star_B_maghitude,center_star_V_maghitude,Messier_number,NGC_numner,IC_number,Center_star_name,Identifiers,Common_names,NED_notes,OpenNGC_notes
4923,IC4715,*Ass,18:16:56.12,-18:30:52.4,Sgr,120.0,60.0,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,Small Sgr Star Cloud,Milky Way star cloud.,V-mag taken from HEASARC's messier table
4933,IC4725,OCl,18:31:46.77,-19:06:53.8,Sgr,14.1,NaN,NaN,5.29,4.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


*Note:* Les objets M24 et  M25 existent dans NGC_mattiaAverga_GitHub. Mais M40 et M45 n'y sont pas. Ils sont dans l'Addendum.

### Addendum
Le fichier Addendum contient des objets qui ne font partie des catalogue NGC ou IC mais qui peuvent interessé les amateurs de l'astronomie. Les objets M40 et M45 du catalogue de Messier n'ont pas d'identifiant NGC donc ils sont présents dans cet addendum (voir [source](https://github.com/mattiaverga/OpenNGC/blob/master/README.md)).

In [3]:
# Lecture de données depuis le fichier source vers un objet DataFrame
addendum_NGC_mattiaAverga_GitHub = pd.read_csv(os.path.join(CHEMIN_SOURCES, "addendum_NGC_mattiaAverga_GitHub.csv"), sep=';', header=0, names=col_names)
addendum_NGC_mattiaAverga_GitHub.shape

(20, 26)

## 2. Construction du catalogue de Messier
Dans ce qui suit, nous allons construire notre catalogue de Messier à partir de plusieurs sources.

### 2.1. Le catalogue de Messier: *'Datastro'*
[Source](https://www.datastro.eu/explore/dataset/catalogue-de-messier/table/?disjunctive.objet&disjunctive.mag&disjunctive.english_name_nom_en_anglais&disjunctive.french_name_nom_francais&disjunctive.latin_name_nom_latin&sort=messier)

#### 2.1.1. Lecture du fichier

In [4]:
# Lecture de données depuis le fichier source vers un objet DataFrame
col_names = ['Messier_number',
    'NGC_IC_designation', 
    'object_type',
    'Season',
    'Magnitude',
    'Constellation_EN',
    'Constellation_FR',
    'Constellation_Latin',
    'Right_Ascension',        
    'Declinaison',
    'Distance_light_year',
    'Size',
    'Discoverer',
    'Year',
    'Image1',
    'Image2',
    'Constelation_abr' ]
messier_Datastro_website = pd.read_csv(os.path.join(CHEMIN_SOURCES, "messier_Datastro_website.csv"), sep=';', header=0, names=col_names)
messier_Datastro_website.shape

(110, 17)

In [5]:
# Corrections d'un identifiant NGC 
messier_Datastro_website.loc[messier_Datastro_website['NGC_IC_designation']=='NGC 9176', 'NGC_IC_designation'] = 'NGC 1976'

#### 2.1.2. Division du catalogue de Messier

Nous avons besoin de traiter à part les deux objets M40 et M45. Car ces deux objets vont être mergés avec les données présentes dans le *Addendum* et le reste des objets Messier vont être mergés avec les données du *NGC_mattiaAverga_GitHub*.

In [7]:
messier_Datastro_website_without_M40_M45 = messier_Datastro_website[(messier_Datastro_website['Messier_number']!='M40') &
                        (messier_Datastro_website['Messier_number']!='M45')].copy()

messier_Datastro_website_M40_M45 = messier_Datastro_website[(messier_Datastro_website['Messier_number']=='M40') |
                        (messier_Datastro_website['Messier_number']=='M45')].copy()

#### 2.1.3. Merging entre Messier et NGC

In [38]:
# Normalisation du NGC_designation: NGCnnnn (suppression des vides)
messier_Datastro_website_without_M40_M45.loc[messier_Datastro_website_without_M40_M45['NGC_IC_designation'].str.len()==8
                             , 'NGC_IC_designation']= messier_Datastro_website_without_M40_M45['NGC_IC_designation'].str.replace(" ", "")

messier_Datastro_website_without_M40_M45.loc[messier_Datastro_website_without_M40_M45['NGC_IC_designation'].str.len()==7
                             , 'NGC_IC_designation']= messier_Datastro_website_without_M40_M45['NGC_IC_designation'].str.replace(" ", "0")

In [39]:
# Merge du dataframe qui contient tout les objets sauf M40 et M45
merge_messier_Datastro_website_without_M40_M45 = messier_Datastro_website_without_M40_M45.merge(NGC_mattiaAverga_GitHub, how='left', on='NGC_IC_designation')
merge_messier_Datastro_website_without_M40_M45.shape

(108, 42)

*Liste des features:*

- Messier_number_x
- NGC_IC_designation
- *object_type_x* (delete object_type_y is abreviation) 
- Season                         
- Magnitude                       
- Constellation_EN               
- Constellation_FR               
- Constellation_Latin            
- Right_Ascension_x  1 Nan(keep Right_Ascension_y no nan)            
- Declinaison_x  1 Nan(keep Declinaison_y no nan)                
- Distance_light_year           
- Size                           
- Discoverer                     
- Year                          
- Image1                         
- Image2                         
- *Constelation_abr (Delete)*             
- *object_type_y  (delete)*            
- *Right_Ascension_y*             
- *Declinaison_y*                  
- Constelation_name_abrv      
- major_axis                   
- minor_axis                    
- major__axis_position_angle    
- B_Apparent_Magnitude          
- V_Apparent_Magnitude          
- J_Apparent_Magnitude          
- H_Apparent_Magnitude          
- K_Apparent_Magnitude          
- Mean_surface_brigthness       
- Hubble_morphological_type      
- center_star_U_maghitude       
- center_star_B_maghitude       
- center_star_V_maghitude       
- *Messier_number_y (delete)*             
- NGC_numner  (6 double-listed in NGC)                  
- *IC_number (delete: 108 Nan)*                    
- Center_star_name               
- Identifiers                    
- Common_names                   
- NED_notes                      
- OpenNGC_notes

In [40]:
# Suppression des colunne inutiles ou redondantes
drop_list = ['object_type_y', 'Right_Ascension_x', 'Declinaison_x', 'Messier_number_y', 'IC_number']
rename_lis = {'Messier_number_x' : 'Messier_number',
             'object_type_x' : 'object_type',
             'Right_Ascension_y' : 'Right_Ascension',
             'Declinaison_y' : 'Declinaison'}

merge_messier_Datastro_website_without_M40_M45 = merge_messier_Datastro_website_without_M40_M45.drop(columns=drop_list)
merge_messier_Datastro_website_without_M40_M45 = merge_messier_Datastro_website_without_M40_M45.rename(columns=rename_lis)
merge_messier_Datastro_website_without_M40_M45.shape

(108, 37)

In [11]:
# Extraction des objets M40 et M45 depuis l'Addendum
Open_NGC_M40_M45 = addendum_NGC_mattiaAverga_GitHub[~addendum_NGC_mattiaAverga_GitHub['Messier_number'].isna()].copy()
# Ajout du Numero Messier standardisé
Open_NGC_M40_M45['Messier_number_bis'] = 'M'+Open_NGC_M40_M45['Messier_number'].astype('int').astype('str')

# Remplacement de la colonne Messier_number par la nouvelle colonne
Open_NGC_M40_M45 = Open_NGC_M40_M45.drop(columns='Messier_number')
Open_NGC_M40_M45 = Open_NGC_M40_M45.rename(columns={'Messier_number_bis': 'Messier_number'})
Open_NGC_M40_M45

,NGC_IC_designation,object_type,Right_Ascension,Declinaison,Constelation_name_abrv,major_axis,minor_axis,major__axis_position_angle,B_Apparent_Magnitude,V_Apparent_Magnitude,J_Apparent_Magnitude,H_Apparent_Magnitude,K_Apparent_Magnitude,Mean_surface_brigthness,Hubble_morphological_type,center_star_U_maghitude,center_star_B_maghitude,center_star_V_maghitude,NGC_numner,IC_number,Center_star_name,Identifiers,Common_names,NED_notes,OpenNGC_notes,Messier_number
13,M040,**,12:22:16.1,+58:05:04,UMa,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WDS J12222+5805AB,NaN,NaN,VMag taken from HEASARC's messier table,M40
14,Mel022,OCl,03:47:28.6,+24:06:19,Tau,150.0,150.0,90.0,NaN,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWSC 0305,Pleiades,NaN,"Coordinates taken from Simbad, VMag taken from...",M45


In [12]:
# Merge du dataframe qui contient M40 et M45
merge_messier_Datastro_website_M40_M45 = messier_Datastro_website_M40_M45.merge(Open_NGC_M40_M45, how='left', on='Messier_number')
merge_messier_Datastro_website_M40_M45

,Messier_number,NGC_IC_designation_x,object_type_x,Season,Magnitude,Constellation_EN,Constellation_FR,Constellation_Latin,Right_Ascension_x,Declinaison_x,Distance_light_year,Size,Discoverer,Year,Image1,Image2,Constelation_abr,NGC_IC_designation_y,object_type_y,Right_Ascension_y,Declinaison_y,Constelation_name_abrv,major_axis,minor_axis,major__axis_position_angle,B_Apparent_Magnitude,V_Apparent_Magnitude,J_Apparent_Magnitude,H_Apparent_Magnitude,K_Apparent_Magnitude,Mean_surface_brigthness,Hubble_morphological_type,center_star_U_maghitude,center_star_B_maghitude,center_star_V_maghitude,NGC_numner,IC_number,Center_star_name,Identifiers,Common_names,NED_notes,OpenNGC_notes
0,M45,NaN,Open Cluster / Amas Ouvert,Winter / Hiver,1,NaN,NaN,NaN,NaN,NaN,410.0,"120,0'",NaN,NaN,http://www.lasam.ca/messier/M045.JPG,https://www.datastro.eu/api/v2/catalog/dataset...,NaN,Mel022,OCl,03:47:28.6,+24:06:19,Tau,150.0,150.0,90.0,NaN,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWSC 0305,Pleiades,NaN,"Coordinates taken from Simbad, VMag taken from..."
1,M40,Winnecke 4,Double star / Étoile Double,Spring / Printemps,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hevelius,1660.0,http://www.lasam.ca/messier/M040.JPG,https://www.datastro.eu/api/v2/catalog/dataset...,NaN,M040,**,12:22:16.1,+58:05:04,UMa,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WDS J12222+5805AB,NaN,NaN,VMag taken from HEASARC's messier table


In [13]:
# Move data before deleting NGC_IC_designation_y
merge_messier_Datastro_website_M40_M45.loc[merge_messier_Datastro_website_M40_M45['NGC_IC_designation_x'].isna(),
                                          'NGC_IC_designation_x'] = merge_messier_Datastro_website_M40_M45['NGC_IC_designation_y']

In [14]:
# Suppression des colunne inutiles ou redondantes
drop_list = ['NGC_IC_designation_y', 'object_type_y', 'Right_Ascension_x', 'Declinaison_x', 'IC_number']
rename_lis = {'NGC_IC_designation_x': 'NGC_IC_designation',
             'object_type_x' : 'object_type',
             'Right_Ascension_y' : 'Right_Ascension',
             'Declinaison_y' : 'Declinaison'}

merge_messier_Datastro_website_M40_M45 = merge_messier_Datastro_website_M40_M45.drop(columns=drop_list)
merge_messier_Datastro_website_M40_M45 = merge_messier_Datastro_website_M40_M45.rename(columns=rename_lis)
merge_messier_Datastro_website_M40_M45.shape

(2, 37)

#### 2.1.4. Concatination entre Messier contenant les objets M40 et M45 et le Messier ayant le reste des objets

In [15]:
messier_catalogue = pd.concat([merge_messier_Datastro_website_without_M40_M45, merge_messier_Datastro_website_M40_M45])

In [16]:
messier_catalogue[messier_catalogue['Hubble_morphological_type'].isna()]

,Messier_number,NGC_IC_designation,object_type,Season,Magnitude,Constellation_EN,Constellation_FR,Constellation_Latin,Distance_light_year,Size,Discoverer,Year,Image1,Image2,Constelation_abr,Right_Ascension,Declinaison,Constelation_name_abrv,major_axis,minor_axis,major__axis_position_angle,B_Apparent_Magnitude,V_Apparent_Magnitude,J_Apparent_Magnitude,H_Apparent_Magnitude,K_Apparent_Magnitude,Mean_surface_brigthness,Hubble_morphological_type,center_star_U_maghitude,center_star_B_maghitude,center_star_V_maghitude,NGC_numner,Center_star_name,Identifiers,Common_names,NED_notes,OpenNGC_notes
2,M76,NGC0650,Planetary Nebula / Nébuleuse Planétaire,Autumn / Automne,10,Perseus,Persée,Perseus,8200.0,65'',Méchain,1780.0,http://www.lasam.ca/messier/M076.JPG,https://www.datastro.eu/api/v2/catalog/dataset...,Per,01:42:19.69,+51:34:31.7,Per,1.12,NaN,NaN,12.20,10.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.48,0651,NaN,"2MASX J01421808+5134243,IRAS 01391+5119,PN G13...","Barbell Nebula,Cork Nebula,Little Dumbbell Nebula",The 2MASX position is centered on a superposed...,NaN
3,M80,NGC6093,Globular Cluster / Amas Globulaire,Summer / Été,7,Scorpion,Scorpion,Scorpius,36000.0,"5,1'",Messier,1781.0,http://www.lasam.ca/messier/M080.JPG,https://www.datastro.eu/api/v2/catalog/dataset...,Sco,16:17:02.51,-22:58:30.4,Sco,5.70,NaN,NaN,NaN,7.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWSC 2376,NaN,NaN,V-mag taken from LEDA
4,M93,NGC2447,Open Cluster / Amas Ouvert,Winter / Hiver,6,"Stern,Poop deck",Poupe,Puppis,3600.0,"22,0'",Messier,1781.0,http://www.lasam.ca/messier/M093.JPG,https://www.datastro.eu/api/v2/catalog/dataset...,Pup,07:44:29.23,-23:51:11.1,Pup,15.00,NaN,NaN,6.57,6.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWSC 1324,NaN,NaN,NaN
7,M37,NGC2099,Open Cluster / Amas Ouvert,Winter / Hiver,5,Charioteer,Cocher,Auriga,3600.0,"24,0'",Hodierna,1654.0,http://www.lasam.ca/messier/M037.JPG,https://www.datastro.eu/api/v2/catalog/dataset...,Aur,05:52:18.35,+32:33:10.8,Aur,11.40,NaN,NaN,6.19,5.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWSC 0689,NaN,NaN,NaN
8,M36,NGC1960,Open Cluster / Amas Ouvert,Winter / Hiver,6,Charioteer,Cocher,Auriga,3700.0,"12,0'",Hodierna,1654.0,http://www.lasam.ca/messier/M036.JPG,https://www.datastro.eu/api/v2/catalog/dataset...,Aur,05:36:17.74,+34:08:26.7,Aur,7.20,NaN,NaN,6.09,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWSC 0594,NaN,NaN,NaN
9,M48,NGC2548,Open Cluster / Amas Ouvert,Winter / Hiver,5,"Hydra, Sea Snake",Hydre,Hydra,1500.0,42',Messier,1771.0,http://www.lasam.ca/messier/M048.JPG,https://www.datastro.eu/api/v2/catalog/dataset...,Hya,08:13:43.18,-05:45:01.6,Hya,28.20,NaN,NaN,6.11,5.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWSC 1454,NaN,NaN,NaN
14,M16,NGC6611,Emission Nebula / Nébuleuse à émission,Summer / Été,6,NaN,NaN,NaN,5870.0,"25,0'",Chéseaux,1746.0,http://www.lasam.ca/messier/M016.JPG,https://www.datastro.eu/api/v2/catalog/dataset...,Se2,18:18:48.17,-13:48:26.0,Se2,120.00,25.0,NaN,6.58,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LBN 67,MWSC 2886",Eagle Nebula,NaN,NaN
15,M4,NGC6121,Globular Cluster / Amas Globulaire,Summer / Été,5,Scorpion,Scorpion,Scorpius,9900.0,"22,8'",Chéseaux,1746.0,http://www.lasam.ca/messier/M004.JPG,https://www.datastro.eu/api/v2/catalog/dataset...,Sco,16:23:35.40,-26:31:31.9,Sco,28.20,NaN,NaN,NaN,5.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWSC 2396,NaN,NaN,V-mag taken from LEDA
16,M15,NGC7078,Globular Cluster / Amas Globulaire,Autumn / Automne,6,Winged Horse,Pégase,Pegasus,49500.0,"12,3'",Maraldi,1746.0,http://www.lasam.ca/messier/M015.JPG,https://www.datastro.eu/api/v2/catalog/dataset...,Peg,21:29:58.38,+12:10:00.6,Peg,11.10,NaN,NaN,3.00,6.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"IRAS 21274+1156,MWSC 3518,PN G065.0-27.3",NaN,NaN,V-mag taken from LEDA
17,M18,NGC6613,Open Cluster / Amas Ouvert,Summer / Été,6,Archer,Sagittaire,Sagittarius,4900.0,"9,0'",Messier,1764.0,http://www.lasam.ca/messier/M018.JPG,https://www.datastro.eu/api/v2/catalog/dataset...,Sgr,18:19:58.49,-17:06:07.1,Sgr,6.00,NaN,NaN,7

### 2.3. Le catalogue de Messier: *'jbcurtin_gitHub'*
Ce catalogue ([Voir référence](https://github.com/jbcurtin/messier-catalogue)) est pricipalement intéressant pour les images qu'il contient pour chaque objet.

In [17]:
# Lecture de données depuis le fichier source vers un objet DataFrame
col_names = ['Messier_number_jbcurtin_gitHub',
    'NGC_IC_designation_jbcurtin_gitHub', 
    'Common_name_jbcurtin_gitHub',
    'image_jbcurtin_gitHub',
    'Object_type_jbcurtin_gitHub',
    'Distance_KLY_jbcurtin_gitHub',             
    'Constellation_jbcurtin_gitHub',
    'Apparent_magnitude_jbcurtin_gitHub',
    'Right_Ascension_jbcurtin_gitHub',        
    'Declinaison_jbcurtin_gitHub']
messier_with_picture_links_jbcurtin_gitHub = pd.read_csv(os.path.join(CHEMIN_SOURCES, "messier_with_picture_links_jbcurtin_gitHub.csv"), keep_default_na=True, sep=';', header=0, names=col_names)
messier_with_picture_links_jbcurtin_gitHub.loc[messier_with_picture_links_jbcurtin_gitHub['Common_name_jbcurtin_gitHub']==' ',
                       'Common_name_jbcurtin_gitHub'] = np.nan #not a space
    
messier_with_picture_links_jbcurtin_gitHub.dtypes

Messier_number_jbcurtin_gitHub         object
NGC_IC_designation_jbcurtin_gitHub     object
Common_name_jbcurtin_gitHub            object
image_jbcurtin_gitHub                  object
Object_type_jbcurtin_gitHub            object
Distance_KLY_jbcurtin_gitHub           object
Constellation_jbcurtin_gitHub          object
Apparent_magnitude_jbcurtin_gitHub    float64
Right_Ascension_jbcurtin_gitHub        object
Declinaison_jbcurtin_gitHub            object
dtype: object

In [18]:
# Merge entre 'messier_catalogue' et le nouveau dataset
messier_catalogue2 = messier_with_picture_links_jbcurtin_gitHub.merge(messier_catalogue,how='left', left_on='Messier_number_jbcurtin_gitHub', right_on='Messier_number')
messier_catalogue2.shape

(110, 47)

In [19]:
#keep 'Common_name_jbcurtin_gitHub', delete_Common_names
messier_catalogue2.loc[messier_catalogue2['Common_name_jbcurtin_gitHub'].isna(),
                   'Common_name_jbcurtin_gitHub'] = messier_catalogue2['Common_names']

# In 'Object_type_jbcurtin_gitHub', Galaxy is subtyped. So we should keep that info
# Keep 'object_type', delete 'Object_type_jbcurtin_gitHub'
messier_catalogue2.loc[messier_catalogue2['object_type'].str.contains('Galaxy'),
                   'object_type'] = messier_catalogue2['Object_type_jbcurtin_gitHub']

#Delete 'Constellation_jbcurtin_gitHub'
messier_catalogue2.loc[messier_catalogue2['Constellation_Latin'].isna(),
                   'Constellation_Latin'] = messier_catalogue2['Constellation_jbcurtin_gitHub']

drop_list = ['Common_names', 'Messier_number_jbcurtin_gitHub', 'NGC_IC_designation_jbcurtin_gitHub', 
             'Object_type_jbcurtin_gitHub', 'Distance_KLY_jbcurtin_gitHub', 'Constellation_jbcurtin_gitHub',
            'Right_Ascension_jbcurtin_gitHub', 'Declinaison_jbcurtin_gitHub']
rename_lis = {'Common_name_jbcurtin_gitHub': 'Common_name'}

messier_catalogue2 = messier_catalogue2.drop(columns=drop_list)
messier_catalogue2 = messier_catalogue2.rename(columns=rename_lis)
messier_catalogue2.shape


(110, 39)

### 2.4. Le catalogue de Messier: *'Nexstarsite'*

https://www.nexstarsite.com/Book/DSO.htm

In [20]:
MessierObjects_nexstarsite = pd.read_excel(os.path.join(CHEMIN_SOURCES, "nexstarsite", "MessierObjects_nexstarsite.xls"), skiprows=2)
MessierObjects_nexstarsite.dtypes

ObjectNum          int64
Name              object
Type              object
Constellation     object
RAHour             int64
RAMinute         float64
DecSign           object
DecDeg             int64
DecMinute        float64
Magnitude         object
Info              object
Distance (ly)      int64
dtype: object

Ce dataset ne contient pas de nouvelles données.

In [21]:
messier_catalogue_final = messier_catalogue2

In [22]:
messier_catalogue_final.dtypes

Common_name                            object
image_jbcurtin_gitHub                  object
Apparent_magnitude_jbcurtin_gitHub    float64
Messier_number                         object
NGC_IC_designation                     object
object_type                            object
Season                                 object
Magnitude                               int64
Constellation_EN                       object
Constellation_FR                       object
Constellation_Latin                    object
Distance_light_year                   float64
Size                                   object
Discoverer                             object
Year                                  float64
Image1                                 object
Image2                                 object
Constelation_abr                       object
Right_Ascension                        object
Declinaison                            object
Constelation_name_abrv                 object
major_axis                        

## 3. Construction du catalogue de Caldwell

In [73]:
# Lecture de données depuis le fichier source vers un objet DataFrame
col_names = ['Caldwell_number',
    'NGC_IC_designation', 
    'object_type',
    'Constelation_abr',
    'Magnitude',
    'Size',
    'Right_Ascension_Hour',
    'Right_Ascension_Minute', 
    'Declinaison_Sign',
    'Declinaison_Deg',
    'Declinaison_Minute'
     ]
#messier_with_picture_links_jbcurtin_gitHub = pd.read_csv(os.path.join(CHEMIN_SOURCES, "messier_with_picture_links_jbcurtin_gitHub.csv"), keep_default_na=True, sep=';', header=0, names=col_names)

CaldwellObjects_nexstarsite = pd.read_excel(os.path.join(CHEMIN_SOURCES, "nexstarsite", "CaldwellObjects_nexstarsite.xls"), skiprows=2, keep_default_na=True, names=col_names)
CaldwellObjects_nexstarsite.shape

(109, 11)

In [74]:
CaldwellObjects_nexstarsite

,Caldwell_number,NGC_IC_designation,object_type,Constelation_abr,Magnitude,Size,Right_Ascension_Hour,Right_Ascension_Minute,Declinaison_Sign,Declinaison_Deg,Declinaison_Minute
0,1,NGC 188,Open Cluster,Cep,8.1,Size: 14,0,44.4,+,85,20
1,2,NGC 40,Planetary Nebula,Cep,12.4,Size: 0.6,0,13.0,+,72,32
2,3,NGC 4236,Spiral Galaxy,Dra,9.7,Size: 19 x 7,12,16.7,+,69,28
3,4,NGC 7023,Bright Nebula,Cep,--,Size: 18 x 18,21,1.8,+,68,12
4,5,IC 342,Spiral Galaxy,Cam,9.2,Size: 18 x 17,3,46.8,+,68,6
5,6,NGC 6543,Planetary Nebula,Dra,8.1,Size: 0.3/5.8,17,58.6,+,66,38
6,7,NGC 2403,Spiral Galaxy,Cam,8.4,Size: 18 x 10,7,36.9,+,65,36
7,8,NGC 559,Open Cluster,Cas,9.5,Size: 4,1,29.5,+,63,18
8,9,Sh2-155,Bright Nebula,Cep,--,Size: 50 x 10,22,56.8,+,62,37
9,10,NGC 663,Open Cluster,Cas,7.1,Size: 16,1,46.0,+,61,15


In [61]:
NGC_mattiaAverga_GitHub[NGC_mattiaAverga_GitHub['NGC_IC_designation'] =="NGC0104"]

,NGC_IC_designation,object_type,Right_Ascension,Declinaison,Constelation_name_abrv,major_axis,minor_axis,major__axis_position_angle,B_Apparent_Magnitude,V_Apparent_Magnitude,J_Apparent_Magnitude,H_Apparent_Magnitude,K_Apparent_Magnitude,Mean_surface_brigthness,Hubble_morphological_type,center_star_U_maghitude,center_star_B_maghitude,center_star_V_maghitude,Messier_number,NGC_numner,IC_number,Center_star_name,Identifiers,Common_names,NED_notes,OpenNGC_notes
5724,NGC0104,GCl,00:24:05.36,-72:04:53.2,Tuc,31.8,NaN,NaN,5.78,4.09,2.29,1.76,1.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2MASX J00240535-7204531,C 106,MWSC 0038",47 Tuc Cluster,NaN,NaN


In [58]:
addendum_NGC_mattiaAverga_GitHub[addendum_NGC_mattiaAverga_GitHub['NGC_IC_designation'] =="NGC2070"]

,NGC_IC_designation,object_type,Right_Ascension,Declinaison,Constelation_name_abrv,major_axis,minor_axis,major__axis_position_angle,B_Apparent_Magnitude,V_Apparent_Magnitude,J_Apparent_Magnitude,H_Apparent_Magnitude,K_Apparent_Magnitude,Mean_surface_brigthness,Hubble_morphological_type,center_star_U_maghitude,center_star_B_maghitude,center_star_V_maghitude,Messier_number,NGC_numner,IC_number,Center_star_name,Identifiers,Common_names,NED_notes,OpenNGC_notes


In [48]:
NGC_mattiaAverga_GitHub

,NGC_IC_designation,object_type,Right_Ascension,Declinaison,Constelation_name_abrv,major_axis,minor_axis,major__axis_position_angle,B_Apparent_Magnitude,V_Apparent_Magnitude,J_Apparent_Magnitude,H_Apparent_Magnitude,K_Apparent_Magnitude,Mean_surface_brigthness,Hubble_morphological_type,center_star_U_maghitude,center_star_B_maghitude,center_star_V_maghitude,Messier_number,NGC_numner,IC_number,Center_star_name,Identifiers,Common_names,NED_notes,OpenNGC_notes
0,IC0001,**,00:08:27.05,+27:43:03.6,Peg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IC0002,G,00:11:00.88,-12:49:22.3,Cet,0.98,0.32,142.0,15.46,NaN,12.26,11.48,11.17,23.45,Sb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2MASX J00110081-1249206,IRAS 00084-1306,MCG -0...",NaN,NaN,B-Mag taken from LEDA.
2,IC0003,G,00:12:06.09,-00:24:54.8,Psc,0.93,0.67,53.0,15.10,NaN,11.53,10.79,10.54,23.50,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2MASX J00120604-0024543,MCG +00-01-038,PGC 000...",NaN,NaN,NaN
3,IC0004,G,00:13:26.94,+17:29:11.2,Peg,1.17,0.84,12.0,14.20,NaN,11.51,10.65,10.50,23.01,Sc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2MASX J00132695+1729111,IRAS 00108+1712,MCG +0...",NaN,NaN,NaN
4,IC0005,G,00:17:34.93,-09:32:36.1,Cet,0.99,0.66,9.0,14.57,NaN,11.50,10.85,10.50,23.40,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2MASX J00173495-0932364,MCG -02-01-047,PGC 001...",NaN,NaN,B-Mag taken from LEDA.
5,IC0006,G,00:18:55.04,-03:16:33.9,Psc,1.23,1.08,146.0,14.50,NaN,11.03,10.32,10.08,23.89,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2MASX J00185505-0316339,MCG -01-01-075,PGC 001228",NaN,NaN,NaN
6,IC0007,G,00:18:53.16,+10:35:40.9,Psc,0.90,0.63,174.0,14.70,NaN,11.33,10.57,10.26,23.22,S0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2MASX J00185316+1035410,PGC 001216",NaN,NaN,NaN
7,IC0008,G,00:19:02.72,-03:13:19.5,Psc,0.82,0.34,129.0,15.16,NaN,12.70,12.08,12.08,23.40,E?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2MASX J00190272-0313196,MCG -01-01-076,PGC 001234",NaN,NaN,B-Mag taken from LEDA.
8,IC0009,G,00:19:43.98,-14:07:18.8,Cet,0.59,0.46,122.0,15.41,NaN,12.38,11.71,11.28,22.88,Sa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2MASX J00194400-1407184,MCG -02-02-001,PGC 001271",NaN,NaN,B-Mag taken from LEDA.
9,IC0010,G,00:20:17.34,+59:18:13.6,Cas,6.76,6.03,129.0,13.60,9.50,7.23,6.34,6.01,24.53,IB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2MASX J00201733+5918136,IRAS 00175+5902,IRAS 0...",NaN,The 2MASX position refers to the center of the...,NaN


In [ ]:
# Normalisation du NGC_designation: NGCnnnn (suppression des vides)
CaldwellObjects_nexstarsite.loc[CaldwellObjects_nexstarsite['NGC_IC_designation'].str.len()==8
                             , 'NGC_IC_designation']= CaldwellObjects_nexstarsite['NGC_IC_designation'].str.replace(" ", "")

CaldwellObjects_nexstarsite.loc[CaldwellObjects_nexstarsite['NGC_IC_designation'].str.len()==7
                             , 'NGC_IC_designation']= CaldwellObjects_nexstarsite['NGC_IC_designation'].str.replace(" ", "0")

## 4. Construction du catalogue de Herschel400

In [24]:
Herschel400_nexstarsite = pd.read_excel(os.path.join(CHEMIN_SOURCES, "nexstarsite","Herschel400_nexstarsite.xls"), skiprows=2)
Herschel400_nexstarsite.shape

(400, 11)

# Catalogues Merging

## Final Catalogue file

In [29]:
reorder = ['Messier_number', 'NGC_IC_designation', 'object_type', 'Common_name', 'Constellation_FR', 'Constellation_EN',
'Constellation_Latin', 'Constelation_abr','Constelation_name_abrv','Hubble_morphological_type', 'Discoverer', 'Year', 
'Season', 'Right_Ascension','Declinaison', 'Distance_light_year', 'Size', 'major_axis', 'minor_axis', 'major__axis_position_angle',
'Apparent_magnitude_jbcurtin_gitHub', 'Magnitude', 'B_Apparent_Magnitude', 'V_Apparent_Magnitude', 'J_Apparent_Magnitude',
'H_Apparent_Magnitude', 'K_Apparent_Magnitude', 'Mean_surface_brigthness',
'Center_star_name', 'center_star_U_maghitude', 'center_star_B_maghitude', 'center_star_V_maghitude',
'Image1', 'Image2', 'image_jbcurtin_gitHub', 'NGC_numner', 'Identifiers', 'NED_notes', 'OpenNGC_notes']

messier_catalogue_final = messier_catalogue_final[reorder]

In [35]:
#messier_catalogue_final.to_csv(os.path.join(CHEMIN_OUTPUT, "Final_catalogue.csv"),";")
messier_catalogue_final.to_csv("Final_catalogue.csv",";")